In [1]:
# Objective du projet : Faire un beau projet qui mêle du Python et du SQL 
#(par exemple par un connecteur de données ou bien l'écriture de requêtes SQL puis l'export des résultats dans une base SQL)

In [1]:
# Import des packages utiles pour l'étude de cas
import os
import mysql.connector
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

ModuleNotFoundError: No module named 'mysql'

In [4]:
# Source des données : http://insideairbnb.com/get-the-data

# Paris data
Paris_reviews = pd.read_csv('C:/Users/margo/Documents/Documents/Formation/Github/0_Data/Airbnb_Paris/reviews.csv')
Paris_listings = pd.read_csv('C:/Users/margo/Documents/Documents/Formation/Github/0_Data/Airbnb_Paris/listings.csv')
Paris_calendar = pd.read_csv('C:/Users/margo/Documents/Documents/Formation/Github/0_Data/Airbnb_Paris/calendar.csv')

# Singapore data
SG_reviews = pd.read_csv('C:/Users/margo/Documents/Documents/Formation/Github/0_Data/Airbnb_Singapore/reviews.csv')
SG_listings = pd.read_csv('C:/Users/margo/Documents/Documents/Formation/Github/0_Data/Airbnb_Singapore/listings.csv')
SG_calendar = pd.read_csv('C:/Users/margo/Documents/Documents/Formation/Github/0_Data/Airbnb_Singapore/calendar.csv')

In [56]:
# Les données sont trop volumineuses pour pouvoir être envoyées sur MySQL Workbench
#Methodo : Je vais prendre un échantillon de données, exporter le csv et ensuite l'ingérer manuellement sous Workbench
SG_reviews_short = SG_reviews[SG_reviews["date"]>= '2022-01-01'] 
SG_listings_short = SG_listings[SG_listings["last_scraped"]>= '2022-01-01']
SG_calendar_short = SG_calendar[SG_calendar["date"]>= '2022-01-01']

SG_listings_short.to_csv("SG_listings_short.csv", sep=';', index = False)
SG_reviews_short.to_csv("SG_reviews_short.csv", sep=';', index = False)
SG_calendar_short.to_csv("SG_calendar_short.csv", sep=';', index = False)
#SG_listings_short = SG_listings_short.drop(['name', 'description', 'neighborhood_overview', 'host_about', "amenities"], axis=1)


In [55]:
SG_listings_short

,id,listing_url,scrape_id,last_scraped,source,picture_url,host_id,host_url,host_name,host_since,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,50646,https://www.airbnb.com/rooms/50646,20220922191314,2022-09-23,previous scrape,https://a0.muscache.com/pictures/36342984/670d...,227796,https://www.airbnb.com/users/show/227796,Sujatha,2010-09-08,...,4.94,4.72,4.50,NaN,f,1,0,1,0,0.18
1,71609,https://www.airbnb.com/rooms/71609,20220922191314,2022-09-22,city scrape,https://a0.muscache.com/pictures/24453191/3580...,367042,https://www.airbnb.com/users/show/367042,Belinda,2011-01-29,...,4.78,4.26,4.32,NaN,f,6,0,6,0,0.15
2,71896,https://www.airbnb.com/rooms/71896,20220922191314,2022-09-22,city scrape,https://a0.muscache.com/pictures/2440674/ac4f4...,367042,https://www.airbnb.com/users/show/367042,Belinda,2011-01-29,...,4.43,4.17,4.04,NaN,t,6,0,6,0,0.18
3,71903,https://www.airbnb.com/rooms/71903,20220922191314,2022-09-22,city scrape,https://a0.muscache.com/pictures/568743/7bc623...,367042,https://www.airbnb.com/users/show/367042,Belinda,2011-01-29,...,4.64,4.50,4.36,NaN,f,6,0,6,0,0.34
4,275344,https://www.airbnb.com/rooms/275344,20220922191314,2022-09-22,city scrape,https://a0.muscache.com/pictures/miso/Hosting-...,1439258,https://www.airbnb.com/users/show/1439258,Kay,2011-11-24,...,4.54,4.62,4.46,S0399,f,44,2,42,0,0.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4156,720000271010003429,https://www.airbnb.com/rooms/720000271010003429,20220922191314,2022-09-22,city scrape,https://a0.muscache.com/pictures/prohost-api/H...,466330896,https://www.airbnb.com/users/show/466330896,Blueground,2022-06-27,...,NaN,NaN,NaN,NaN,t,14,14,0,0,NaN
4157,720000552643956732,https://www.airbnb.com/rooms/720000552643956732,20220922191314,2022-09-22,city scrape,https://a0.muscache.com/pictures/prohost-api/H...,466330896,https://www.airbnb.com/users/show/466330896,Blueground,2022-06-27,...,NaN,NaN,NaN,NaN,t,14,14,0,0,NaN
4158,720695231887902167,https://www.airbnb.com/rooms/720695231887902167,20220922191314,2022-09-22,city scrape,https://a0.muscache.com/pictures/miso/Hosting-...,480404953,https://www.airbnb.com/users/show/480404953,Natividad,2022-09-21,...,NaN,NaN,NaN,Authorised Serviced Apartment,t,1,1,0,0,NaN
4159,720724511357857504,https://www.airbnb.com/rooms/720724511357857504,20220922191314,2022-09-23,city scrape,https://a0.muscache.com/pictures/miso/Hosting-...,158018542,https://www.airbnb.com/users/show/158018542,Betris,2017-11-09,...,NaN,NaN,NaN,NaN,f,4,0,4,0,NaN


In [9]:
# Connection MySQL
user_name = os.environ.get('DB_USER')
password = os.environ.get('DB_PASS')

## MASQUER MES INFOS !!!
connection = mysql.connector.connect(host = 'localhost',
                                    user = 'root',
                                    passwd = 'NuitNoire.12',
                                    db = 'margotdb')
connection

In [10]:
# A partir de maintenant, je fais comme si je ne travaillais pas sur python mais SQL !
cursor = connection.cursor()

query = ("SELECT COUNT(*) FROM Paris_reviews "
        )

#hire_start = datetime.date(1999, 1, 1)
#hire_end = datetime.date(1999, 12, 31)

cursor.execute(query)


ProgrammingError: 1146 (42S02): Table 'margotdb.paris_reviews' doesn't exist

In [ ]:
test = pd.read_sql_query

In [ ]:
# Je fais des opérations sous Python et je vais essayer d'écrire sur MySQL workbench
Paris_reviews.head(10)
test = Paris_reviews[Paris_reviews["date"]>= '2022-09-01']

In [6]:
from google.cloud import bigquery

ImportError: cannot import name 'bigquery' from 'google.cloud' (unknown location)

In [7]:
from google.cloud import bigquery
from google.oauth2 import service_account

ImportError: cannot import name 'bigquery' from 'google.cloud' (unknown location)